In [1]:
storage_name='trangstore'
container_name = 'initiationspark'
destination_filename = 'newtwitter.csv'
access_key='PRsFNIbFLI8+NwtCQdQ4ByhQ50njGG2eLRdcgBt/Ljf8MkyFiaNJo01jZZqzQ38EmIMIoJrp6ERQfD/ZGOoorA=='

inputfile_path = "wasbs://{}@{}.blob.core.windows.net/{}".format(container_name, storage_name, destination_filename)
df = spark.read.format("csv").load(inputfile_path, header=True, inferSchema=True)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2375203317112956> in <module> 
 1 df2 = [ ] 
 2 message = "abpfajfoaz" 
 ----> 3 df2 [ "text" ] = message

 TypeError : list indices must be integers or slices, not str

In [3]:
display(df.head(5))

text,id,pubdate
10 Things Missing In The New Twitter Interface http://bit.ly/bLZXI3 #NewTwitter,2602860537,18536
RT @_NATURALBWINNER OH AND I DONT LIKE THIS #NEWTWITTER SHIT < seeeeeee nobody likes this #newtwitter bullsh** . . eat a dick new twitter !,2602850443,18536
RT @HBO24 yo the #newtwitter is better.. YUPP I SAID IT!<What is new twitter...i m still waitin for su?1 2 fill me in,2602761852,18535
"Aaaaaaaand I have the new twitter! Yay! I should have checked sooner, but I have been busy. I like it! #NewTwitter",2602738438,18535
can I please have the new twitter? #twitter #newtwitter,2602684185,18535


#Text preprocessing

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re
import string

#### Remove non ASCII characters

In [7]:
from pyspark.sql import functions as F

#see the first text 

df.first().text


Out[7]: '10 Things Missing In The New Twitter Interface http://bit.ly/bLZXI3 #NewTwitter'

In [8]:

# remove non ASCII characters
def strip_non_ascii(data_str):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in data_str if 0 < ord(c) < 127)
    return ''.join(stripped)
# setup pyspark udf function
strip_non_ascii_udf = udf(strip_non_ascii, StringType())


In [9]:
df = df.withColumn('text_non_asci',strip_non_ascii_udf(df['text']))
df.show(5,True)

+--------------------+----------+-------+--------------------+
 text| id|pubdate| text_non_asci|
+--------------------+----------+-------+--------------------+
10 Things Missing...|2602860537| 18536|10 Things Missing...|
RT @_NATURALBWINN...|2602850443| 18536|RT @_NATURALBWINN...|
RT @HBO24 yo the ...|2602761852| 18535|RT @HBO24 yo the ...|
Aaaaaaaand I have...|2602738438| 18535|Aaaaaaaand I have...|
can I please have...|2602684185| 18535|can I please have...|
+--------------------+----------+-------+--------------------+
only showing top 5 rows

#### fixed abbreviation

In [11]:
def fix_abbreviation(data_str):
    data_str = data_str.lower()
    data_str = re.sub(r'\bthats\b', 'that is', data_str)
    data_str = re.sub(r'\bive\b', 'i have', data_str)
    data_str = re.sub(r'\bim\b', 'i am', data_str)
    data_str = re.sub(r'\bya\b', 'yeah', data_str)
    data_str = re.sub(r'\bcant\b', 'can not', data_str)
    data_str = re.sub(r'\bdont\b', 'do not', data_str)
    data_str = re.sub(r'\bwont\b', 'will not', data_str)
    data_str = re.sub(r'\bid\b', 'i would', data_str)
    data_str = re.sub(r'wtf', 'what the fuck', data_str)
    data_str = re.sub(r'\bwth\b', 'what the hell', data_str)
    data_str = re.sub(r'\br\b', 'are', data_str)
    data_str = re.sub(r'\bu\b', 'you', data_str)
    data_str = re.sub(r'\bk\b', 'OK', data_str)
    data_str = re.sub(r'\bsux\b', 'sucks', data_str)
    data_str = re.sub(r'\bno+\b', 'no', data_str)
    data_str = re.sub(r'\bcoo+\b', 'cool', data_str)
    data_str = re.sub(r'rt\b', '', data_str)
    data_str = data_str.strip()
    return data_str

fix_abbreviation_udf = udf(fix_abbreviation, StringType())

In [12]:
message = "I like my dog"

def sentiment(text):
  text = strip_non_ascii(text)
  text = fix_abbreviation(text)
  text = remove_features(text)
  text = sentiment_analysis(text)
  sentiment = condition(text)
  return sentiment

sentiment = sentiment(message)
print(sentiment)
  

I like my dog
neutral

In [13]:
strip_non_ascii_udf(message)

Out[29]: Column<b'strip_non_ascii(I like my dog)'>

In [14]:
df = df.withColumn('fixed_abbrev',fix_abbreviation_udf(df['text_non_asci']))
df.show(5,True)

+--------------------+----------+-------+--------------------+--------------------+
 text| id|pubdate| text_non_asci| fixed_abbrev|
+--------------------+----------+-------+--------------------+--------------------+
10 Things Missing...|2602860537| 18536|10 Things Missing...|10 things missing...|
RT @_NATURALBWINN...|2602850443| 18536|RT @_NATURALBWINN...|@_naturalbwinner ...|
RT @HBO24 yo the ...|2602761852| 18535|RT @HBO24 yo the ...|@hbo24 yo the #ne...|
Aaaaaaaand I have...|2602738438| 18535|Aaaaaaaand I have...|aaaaaaaand i have...|
can I please have...|2602684185| 18535|can I please have...|can i please have...|
+--------------------+----------+-------+--------------------+--------------------+
only showing top 5 rows

#### remove irrelevant features

In [16]:
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    # remove unwanted space, *.split() will automatically split on
    # whitespace and discard duplicates, the " ".join() joins the
    # resulting list into one string.
    return " ".join(cleaned_str.split())
# setup pyspark udf function
remove_features_udf = udf(remove_features, StringType())


In [17]:
df = df.withColumn('removed',remove_features_udf(df['fixed_abbrev']))
df.show(5,True)

+--------------------+----------+-------+--------------------+--------------------+--------------------+
 text| id|pubdate| text_non_asci| fixed_abbrev| removed|
+--------------------+----------+-------+--------------------+--------------------+--------------------+
10 Things Missing...|2602860537| 18536|10 Things Missing...|10 things missing...|things missing in...|
RT @_NATURALBWINN...|2602850443| 18536|RT @_NATURALBWINN...|@_naturalbwinner ...|oh and do not lik...|
RT @HBO24 yo the ...|2602761852| 18535|RT @HBO24 yo the ...|@hbo24 yo the #ne...|yo the newtwitter...|
Aaaaaaaand I have...|2602738438| 18535|Aaaaaaaand I have...|aaaaaaaand i have...|aaaaaaaand have t...|
can I please have...|2602684185| 18535|can I please have...|can i please have...|can please have t...|
+--------------------+----------+-------+--------------------+--------------------+--------------------+
only showing top 5 rows

### Sentiment Analysis main function

In [19]:
!pip install -U textblob

In [20]:
from pyspark.sql.types import FloatType

from textblob import TextBlob

def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity

sentiment_analysis_udf = udf(sentiment_analysis , FloatType())

df  = df.withColumn("sentiment_score", sentiment_analysis_udf( df['removed'] ))
df.show(5,True)

+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+
 text| id|pubdate| text_non_asci| fixed_abbrev| removed|sentiment_score|
+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+
10 Things Missing...|2602860537| 18536|10 Things Missing...|10 things missing...|things missing in...| -0.03181818|
RT @_NATURALBWINN...|2602850443| 18536|RT @_NATURALBWINN...|@_naturalbwinner ...|oh and do not lik...| -0.03181818|
RT @HBO24 yo the ...|2602761852| 18535|RT @HBO24 yo the ...|@hbo24 yo the #ne...|yo the newtwitter...| 0.3181818|
Aaaaaaaand I have...|2602738438| 18535|Aaaaaaaand I have...|aaaaaaaand i have...|aaaaaaaand have t...| 0.11818182|
can I please have...|2602684185| 18535|can I please have...|can i please have...|can please have t...| 0.13636364|
+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+
only showing top 5 rows

### Sentiment Classification

In [22]:
def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())


df  = df.withColumn("sentiment", sentiment_udf( df['sentiment_score'] ))
df.show(5,True)

+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+---------+
 text| id|pubdate| text_non_asci| fixed_abbrev| removed|sentiment_score|sentiment|
+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+---------+
10 Things Missing...|2602860537| 18536|10 Things Missing...|10 things missing...|things missing in...| -0.03181818| neutral|
RT @_NATURALBWINN...|2602850443| 18536|RT @_NATURALBWINN...|@_naturalbwinner ...|oh and do not lik...| -0.03181818| neutral|
RT @HBO24 yo the ...|2602761852| 18535|RT @HBO24 yo the ...|@hbo24 yo the #ne...|yo the newtwitter...| 0.3181818| positive|
Aaaaaaaand I have...|2602738438| 18535|Aaaaaaaand I have...|aaaaaaaand i have...|aaaaaaaand have t...| 0.11818182| positive|
can I please have...|2602684185| 18535|can I please have...|can i please have...|can please have t...| 0.13636364| positive|
+--------------------+----------+-------+--------------------+--------------------+--------------------+---------------+---------+
only showing top 5 rows

In [23]:
df.registerTempTable("df_sentiment")

In [24]:
%sql
SELECT *
FROM df_sentiment

text,id,pubdate,text_non_asci,fixed_abbrev,removed,sentiment_score,sentiment
10 Things Missing In The New Twitter Interface http://bit.ly/bLZXI3 #NewTwitter,2602860537,18536,10 Things Missing In The New Twitter Interface http://bit.ly/bLZXI3 #NewTwitter,10 things missing in the new twitter interface http://bit.ly/blzxi3 #newtwitter,things missing in the new twitter interface newtwitter,-0.03181818,neutral
RT @_NATURALBWINNER OH AND I DONT LIKE THIS #NEWTWITTER SHIT < seeeeeee nobody likes this #newtwitter bullsh** . . eat a dick new twitter !,2602850443,18536,RT @_NATURALBWINNER OH AND I DONT LIKE THIS #NEWTWITTER SHIT < seeeeeee nobody likes this #newtwitter bullsh** . . eat a dick new twitter !,@_naturalbwinner oh and i do not like this #newtwitter shit < seeeeeee nobody likes this #newtwitter bullsh** . . eat a dick new twitter !,oh and do not like this newtwitter shit seeeeeee nobody likes this newtwitter bullsh eat dick new twitter,-0.03181818,neutral
RT @HBO24 yo the #newtwitter is better.. YUPP I SAID IT!<What is new twitter...i m still waitin for su?1 2 fill me in,2602761852,18535,RT @HBO24 yo the #newtwitter is better.. YUPP I SAID IT!<What is new twitter...i m still waitin for su?1 2 fill me in,@hbo24 yo the #newtwitter is better.. yupp i said it!<what is new twitter...i m still waitin for su?1 2 fill me in,yo the newtwitter is better yupp said it what is new twitter still waitin for su fill me in,0.3181818,positive
"Aaaaaaaand I have the new twitter! Yay! I should have checked sooner, but I have been busy. I like it! #NewTwitter",2602738438,18535,"Aaaaaaaand I have the new twitter! Yay! I should have checked sooner, but I have been busy. I like it! #NewTwitter","aaaaaaaand i have the new twitter! yay! i should have checked sooner, but i have been busy. i like it! #newtwitter",aaaaaaaand have the new twitter yay should have checked sooner but have been busy like it newtwitter,0.11818182,positive
can I please have the new twitter? #twitter #newtwitter,2602684185,18535,can I please have the new twitter? #twitter #newtwitter,can i please have the new twitter? #twitter #newtwitter,can please have the new twitter twitter newtwitter,0.13636364,positive
"Seriously, still no new Twitter layout? How many weeks is this suppose to take?! #newtwitter",2602678435,18535,"Seriously, still no new Twitter layout? How many weeks is this suppose to take?! #newtwitter","seriously, still no new twitter layout? how many weeks is this suppose to take?! #newtwitter",seriously still no new twitter layout how many weeks is this suppose to take newtwitter,0.032828283,neutral
"""Confieso q el #NewTwitter no me mata, c/vez que veo """"psst the new twitter is here"""" (o como sea que dice) lo ignoro esperando a q desaparezc""",2602600751,18535,"""Confieso q el #NewTwitter no me mata, c/vez que veo """"psst the new twitter is here"""" (o como sea que dice) lo ignoro esperando a q desaparezc""","""confieso q el #newtwitter no me mata, c/vez que veo """"psst the new twitter is here"""" (o como sea que dice) lo ignoro esperando a q desaparezc""",confieso el newtwitter no me mata vez que veo psst the new twitter is here como sea que dice lo ignoro esperando desaparezc,0.13636364,positive
I have the new twitter. Who else? :) #NewTwitter,2602585892,18535,I have the new twitter. Who else? :) #NewTwitter,i have the new twitter. who else? :) #newtwitter,have the new twitter who else newtwitter,0.13636364,positive
HOW DO I DOWNLOAD NEW TWITTER? #newtwitter #help #knowledge #thisISshit,2602548822,18535,HOW DO I DOWNLOAD NEW TWITTER? #newtwitter #help #knowledge #thisISshit,how do i download new twitter? #newtwitter #help #knowledge #thisisshit,how do download new twitter newtwitter help knowledge thisisshit,0.13636364,positive
"same here- urs fixed yet? RT @tamarafarley Hrm, my #lists function doesn't work on the new Twitter. What gives? #newTwitter #fail",2602529998,18535,"same here- urs fixed yet? RT @tamarafarley Hrm, my #lists function doesn't work on th

In [25]:
%sql
SELECT sentiment, count(1)
FROM df_sentiment
GROUP BY 1

sentiment,count(1)
positive,4540
neutral,3043
negative,1011


In [26]:
%sql
SELECT sentiment, count(1)
FROM df_sentiment
GROUP BY 1

sentiment,count(1)
positive,4540
neutral,3043
negative,1011
